# Spatial Orientation Classification

Before deploying our classification model on user-generated videos, we will conduct preliminary tests using our curated dataset. This dataset has been structured and labeled in accordance with the standards outlined in Irina Kartaly's Pole Dance Fitness guidebook, which provides us with a reliable ground truth.

### Required Modules

In [21]:
import numpy as np
import pandas as pd

## Feature Engineering and Data Preprocessing Steps

We have employed the following strategies to prepare our dataset:

1. Feature Engineering: We condensed the landmark points for each key body part into single coordinates by calculating the mean x and y positions. This step reduces the complexity of the data and focuses on the most relevant information for pose classification. - This would be the same feature engineering we would use in user-generated video.

2. Data Cleaning and Labeling: We parsed the filenames to extract categorical labels that serve as descriptors and identifiers for the poses. This process enhances the metadata associated with each pose instance.

3. Data Structuring: We organized the data into a structured DataFrame.

In [33]:
def condense_pole_dictionary(row, x_columns, y_columns):
    x = row[x_columns].values
    y = row[y_columns].values
    
    head_x, head_y = x[0:10].mean(), y[0:10].mean()
    chest_x, chest_y = x[[11, 12]].mean(), y[[11, 12]].mean()        
    hip_x, hip_y = x[23:24].mean(), y[23:24].mean()
    knees_x, knees_y = x[25:26].mean(), y[25:26].mean()
    
    knee_right_x, knee_right_y = x[26], y[26]
    foot_right_x, foot_right_y = x[28], y[28]
    
    #foot_right_x, foot_right_y = x[[28, 30, 32]].mean(), y[[28, 30, 32]].mean()
    
    knee_left_x, knee_left_y = x[25], y[25]
    foot_left_x, foot_left_y = x[27], y[27]
    #foot_left_x, foot_left_y = x[[27, 29, 31]].mean(), y[[27, 29, 31]].mean()

    row['head_x'], row['head_y'] = head_x, head_y
    row['chest_x'], row['chest_y'] = chest_x, chest_y
    row['hip_x'], row['hip_y'] = hip_x, hip_y    
    row['knees_x'], row['knees_y'] = knees_x, knees_y
    
    row['kneeR_x'], row['kneeR_y'] = knee_right_x, knee_right_y
    row['footR_x'], row['footR_y'] = foot_right_x, foot_right_y
    
    row['kneeL_x'], row['kneeL_y'] = knee_left_x, knee_left_y
    row['footL_x'], row['footL_y'] = foot_left_x, foot_left_y
    
    return row

dict = pd.read_csv('dictionary.csv')
x_columns = sorted([col for col in dict.columns if col.endswith('_x')])
y_columns = sorted([col for col in dict.columns if col.endswith('_y')])

# Apply the function to each row
dict = dict.apply(lambda row: condense_pole_dictionary(row, x_columns, y_columns), axis=1)

# Extract 'category' and 'pole_name' from 'filename' and add them as new columns
pattern = r"(?P<category>\w+)-\d+-(?P<name>.+?)(?:\.png)"
extracted_columns = dict['filename'].str.extract(pattern)
extracted_columns['name'] = extracted_columns['name'].str.replace('-', ' ', regex=False)

# Concatenate the new columns with the original DataFrame
dict = pd.concat([dict,extracted_columns], axis=1)
dict

,filename,landmark_00_x,landmark_00_y,landmark_00_z,landmark_00_v,landmark_01_x,landmark_01_y,landmark_01_z,landmark_01_v,landmark_02_x,...,kneeR_x,kneeR_y,footR_x,footR_y,kneeL_x,kneeL_y,footL_x,footL_y,category,name
0,data/external/source-ik/0-horizontal.png,0.698852,0.445517,-0.074413,0.999962,0.705565,0.451498,-0.053239,0.999947,0.706534,...,0.320905,0.462507,0.168074,0.512718,0.333593,0.514175,0.171310,0.543200,NaN,NaN
1,data/external/source-ik/0-inversion.png,0.536107,0.780402,0.107841,0.998548,0.553082,0.771772,0.130719,0.998462,0.552620,...,0.260910,0.362246,0.250732,0.186853,0.269674,0.361198,0.326310,0.207546,NaN,NaN
2,data/external/source-ik/0-stand.png,0.527775,0.201884,-0.668354,0.999997,0.537617,0.189342,-0.626588,0.999991,0.544535,...,0.561001,0.592598,0.606533,0.758664,0.618883,0.601273,0.571576,0.740645,NaN,NaN
3,data/external/source-ik/0-diagonal.png,0.305057,0.453677,-0.772767,0.999846,0.287940,0.441383,-0.766456,0.999431,0.287956,...,0.655235,0.696761,0.766168,0.804233,0.711370,0.524218,0.855968,0.554608,NaN,NaN
4,data/external/source-ik/advanced-1-dakini.png,0.485788,0.272360,0.197538,0.999960,0.472182,0.264307,0.246653,0.999970,0.471197,...,0.550874,0.620815,0.721606,0.753260,0.387084,0.720629,0.323833,0.886837,advanced,dakini
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,data/external/source-ik/workout-50-inverted-pu...,0.476830,0.780403,-0.032172,0.995138,0.480410,0.793465,0.025689,0.993987,0.480092,...,0.591565,0.327102,0.629673,0.166313,0.540887,0.325626,0.623450,0.190052,workout,inverted push ups
571,data/external/source-ik/workout-6-front-kicks.png,0.648227,0.311151,0.047535,0.999820,0.662223,0.296362,0.000055,0.999844,0.668034,...,0.615334,0.742235,0.672226,0.894303,0.431049,0.472117,0.198144,0.400261,workout,front kicks
572,data/external/source-ik/workout-7-pole-diagona...,0.566308,0.329840,-0.437343,0.999365,0.573975,0.310756,-0.419919,0.998598,0.579927,...,0.623623,0.712533,0.675166,0.872327,0.457078,0.712244,0.239937,0.784327,workout,pole diagonal lunges
573,data/external/source-ik/workout-8-pole-side-ki...,0.506299,0.267104,-0.297830,0.996689,0.519483,0.256532,-0.310007,0.993620,0.523483,...,0.331794,0.431762,0.146366,0.358907,0.559435,0.716737,0.571301,0.863647,workout,pole side kicks


## Spatial Orientation Classification

The classification framework for spatial orientation of pole dance poses leverages anatomical and geometric considerations, as substantiated by research in the field of human pose estimation. Drawing upon methodologies such as those described in "Human Pose Estimation with Spatial Contextual Information", our classification system is grounded in the spatial contextual relationships between body landmarks. These relationships—such as the head being positioned above the chest in an upright pose—are not arbitrary but are instead based on established biomechanical patterns that the cited paper's techniques aim to capture and analyze.

By integrating this spatial context into our model, we ensure that the orientation classifications for upright, inverted, horizontal, and diagonal positions reflect the natural human body structure and movement as observed in pole fitness and dance literature. This approach allows for a more nuanced and accurate classification of poses, utilizing both the mean positioning of key body landmarks and the contextual interplay between them to define each pose's orientation.

### Upright Position
- **Condition**: \( head_y > chest_y \)
- **Vertical Alignment**: A vertical alignment is indicated when \( |head_y - chest_y| > 0.05 \), suggesting a significant distance between the head and chest in the vertical axis.

### Horizontal Position
- **Alignment**: \( head_x \approx chest_x \approx hip_x \)
- **Condition**: A horizontal orientation is determined when the difference in x-coordinates is minimal, suggesting alignment along the horizontal axis.

### Diagonal Position
- **Position**: \( head_y > chest_y \)
- **Slope Calculation**: The slope for a diagonal line formed by the legs is calculated as \( slope = \frac{foot2_y - foot1_y}{foot2_x - foot1_x} \), which should not be close to 0 (indicating a horizontal line) or undefined (indicating a vertical line).

### Inverted Position
- **Condition**: \( hip_y > chest_y > head_y \)
- **Ordering**: This ordering of y-coordinates suggests an inverted position relative to the upright stance.

In the `PoseClassifier` class, we implement methods to check for these specific conditions, using the provided landmark coordinates to classify each pose.

### Code Implementation
The `PoseClassifier` class contains methods to assess if a set of y-coordinates are similar within a defined tolerance (horizontal), if the slope between two points suggests a diagonal alignment, and to classify poses based on these assessments. The `classify_row_orientation` function applies these classifications to each row of pose data.


In [69]:
class PoseClassifier:
    def __init__(self, data):
        self.data = data
        self.body_length = self.calculate_body_length()
    
    def calculate_body_length(self):
        head_chest_length = np.sqrt((self.data['head_x'] - self.data['chest_x'])**2 + (self.data['head_y'] - self.data['chest_y'])**2)
        chest_hip_length = np.sqrt((self.data['chest_x'] - self.data['hip_x'])**2 + (self.data['chest_y'] - self.data['hip_y'])**2)
        hip_right_knee_length = np.sqrt((self.data['hip_x'] - self.data['kneeR_x'])**2 + (self.data['hip_y'] - self.data['kneeR_y'])**2)
        right_knee_foot_length = np.sqrt((self.data['kneeR_x'] - self.data['footR_x'])**2 + (self.data['kneeR_y'] - self.data['footR_y'])**2)
        hip_left_knee_length = np.sqrt((self.data['hip_x'] - self.data['kneeL_x'])**2 + (self.data['hip_y'] - self.data['kneeL_y'])**2)
        left_knee_foot_length = np.sqrt((self.data['kneeL_x'] - self.data['footL_x'])**2 + (self.data['kneeL_y'] - self.data['footL_y'])**2)
        right_length = head_chest_length + chest_hip_length + hip_right_knee_length + right_knee_foot_length
        left_length = head_chest_length + chest_hip_length + hip_left_knee_length + left_knee_foot_length
        if right_length > left_length:
            body_length = right_length
        else:
            body_length = left_length
        return body_length
            
    def is_horizontal(self, y_values):
        tolerance = 0.1 * self.body_length  # Dynamic tolerance
        return np.std(y_values) < tolerance

    def angle_between_points(self, x1, y1, x2, y2):
        angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
        return angle

    def is_diagonal(self, x1, y1, x2, y2):
        angle = self.angle_between_points(x1, y1, x2, y2)
        return 30 <= abs(angle) <= 55  # Adjusted to a range around 45 degrees

    def classify_pose(self):
        
        # Extract coordinates
        head_x, head_y = self.data['head_x'], self.data['head_y']
        chest_x, chest_y = self.data['chest_x'], self.data['chest_y']
        hip_x, hip_y = self.data['hip_x'], self.data['hip_y']
        kneeR_x, kneeR_y = self.data['kneeR_x'], self.data['kneeR_y']
        kneeL_x, kneeL_y = self.data['kneeL_x'], self.data['kneeL_y']
        footR_x, footR_y = self.data['footR_x'], self.data['footR_y']
        footL_x, footL_y = self.data['footL_x'], self.data['footL_y']

        # Check for inversion first
        angleR = self.angle_between_points(head_x, head_y, footR_x, footR_y)
        angleL = self.angle_between_points(head_x, head_y, footL_x, footL_y)
        
        if head_y > chest_y > hip_y and (angleR < 0 or angleL < 0):
            return 'inverted'

        # Calculate slopes for potential diagonal lines
        right_side_slope = self.is_diagonal(head_x, head_y, footR_x, footR_y)
        left_side_slope = self.is_diagonal(head_x, head_y, footL_x, footL_y)

        # Check for diagonal pose
        if right_side_slope or left_side_slope:
            return 'diagonal'

        # Check for horizontal pose
        if self.is_horizontal([head_y, chest_y, hip_y]):
            return 'horizontal'

        # Check for upright pose
        if head_y < chest_y < hip_y:
            return 'upright'

        return 'undefined'
        
# Function to classify the row orientation
def classify_row_orientation(row):
    classifier = PoseClassifier(row)
    return classifier.classify_pose()

In [70]:
df = dict.copy()

In [72]:
# The first four images are our quick basis for checking the classification works

df['orientation'] = df.apply(classify_row_orientation, axis=1)
df[['category','filename','orientation']].head(4)

,category,filename,orientation
0,NaN,data/external/source-ik/0-horizontal.png,horizontal
1,NaN,data/external/source-ik/0-inversion.png,inverted
2,NaN,data/external/source-ik/0-stand.png,upright
3,NaN,data/external/source-ik/0-diagonal.png,diagonal


In [73]:
## Save Results
df.to_csv('results2.csv', index=False)

### Implementation on User Data

In [74]:
user_video = pd.read_csv('data/processed/solo-training1/dictionary.csv')
dict = user_video.copy()

x_columns = sorted([col for col in dict.columns if col.endswith('_x')])
y_columns = sorted([col for col in dict.columns if col.endswith('_y')])
dict = dict.apply(lambda row: condense_pole_dictionary(row, x_columns, y_columns), axis=1)
dict['orientation'] = dict.apply(classify_row_orientation, axis=1)
dict[['filename','orientation']]

/var/folders/8q/psx_9spx6sn4p06701_g30nh0000gn/T/ipykernel_41094/1869442712.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dict['orientation'] = dict.apply(classify_row_orientation, axis=1)


,filename,orientation
0,data/processed/solo-training1/00000.png,horizontal
1,data/processed/solo-training1/00006.png,horizontal
2,data/processed/solo-training1/00008.png,horizontal
3,data/processed/solo-training1/00015.png,upright
4,data/processed/solo-training1/00018.png,upright
...,...,...
113,data/processed/solo-training1/00438.png,upright
114,data/processed/solo-training1/00456.png,horizontal
115,data/processed/solo-training1/00458.png,upright
116,data/processed/solo-training1/00465.png,upright


In [75]:
dict.to_csv('data/processed/solo-training1/results.csv', index=False)

## References

* https://github.com/google/mediapipe
* https://www.goodreads.com/book/show/43151907-pole-dance-fitness
* https://ar5iv.labs.arxiv.org/html/1901.01760 